In [1]:
import numpy as np
from scipy.integrate import ode
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

def dxdt_2sv(t, x, par):
    expx = np.exp(x[0])
    yd = (1 - expx) * par['k']
    zd = -expx * par['r'] * (par['b2'] * x[0] + x[2])
    xd = expx * ((par['b1'] - 1) * x[0] + x[1] - x[2]) + yd - zd
    
    return np.array([xd, yd, zd])

In [ ]:


# Define the ODE function (assuming you've already defined dxdt_2sv as in the previous answer)

# Parameters
par = {
    'b1': 1,
    'b2': 0.84,
    'r': 0.048
}

kcr1 = par['b1'] - 1
kcr2 = (kcr1 + par['r'] * (2 * par['b1'] + (par['b2'] - 1) * (2 + par['r'])) + 
        np.sqrt(4 * par['r']**2 * (kcr1 + par['b2']) + (kcr1 + par['r']**2 * (par['b2'] - 1))**2)) / (2 + 2 * par['r'])

sols = pd.DataFrame()
for knd in [0.9, 0.86, 0.856, 0.8552, 0.8525, 0.8, 0.7]:
    print(knd)
    
    par['k'] = knd * kcr2

    # Initial conditions
    x0 = [0.1, 0.1, 0.1]
    # x0 = [np.pi, 0.1, 0.1]

    # Time range
    tmax = 1e5
    tshow = tmax - 2e3

    # Solve ODE
    solver = ode(dxdt_2sv)
    solver.set_integrator('dopri5', atol=1e-13, rtol=1e-7)
    solver.set_initial_value(x0, 0)
    solver.set_f_params(par)

    t = []
    x = []
    dt = 0.1  # time step
    while solver.successful() and solver.t < tmax:
        solver.integrate(solver.t + dt)
        t.append(solver.t)
        x.append(solver.y)

    t = np.array(t)
    x = np.array(x)
    # sols[f't{knd}'] = t
    sols[f'x{knd}'] = x[:,0]
    sols[f'y{knd}'] = x[:,1]
    sols[f'z{knd}'] = x[:,2]

    # # Plot
    # ind = t > tshow
    
    # fig = plt.figure(figsize=(10, 10))
    
    # # y vs. time
    # ax1 = fig.add_subplot(211)
    # ax1.plot(t[ind], x[ind, 1])
    # ax1.set_xlabel('time')
    # ax1.set_ylabel('y')

    # # Phase space
    # ax2 = fig.add_subplot(212, projection='3d')
    # noise = np.random.randn(np.sum(ind), 3) * 0.01
    # ax2.plot3D(x[ind, 0] + noise[:, 0], x[ind, 1] + noise[:, 1], x[ind, 2] + noise[:, 2])
    # ax2.set_xlabel('x')
    # ax2.set_ylabel('y')
    # ax2.set_zlabel('z')

    # # Save data
    # oname = f"p.{knd}.dat"
    # out = np.column_stack((t[ind], x[ind], x[ind] + noise))
    # np.savetxt(oname, out)

    # plt.tight_layout()
    # plt.show()

In [ ]:
sols.filter(regex=('x.*'))[0:5000].plot()

In [ ]:
sols.filter(regex=('y.*'))[0:5000].plot()

In [ ]:
sols.filter(regex=('z.*'))[0:5000].plot()